In [1]:
import numpy as np
import pickle

import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Load t h e p i c k l e f i l e
with open ( 'drive/MyDrive/Kaggle2/train_data.pkl' , 'rb' ) as f :
    data = pickle.load ( f )
# Access images and l a b e l s
images = data ['images']
labels = data ['labels']

In [4]:
images=np.array(images)
labels=np.array(labels)

In [5]:
images.shape

(97477, 28, 28)

In [6]:
labels.shape

(97477,)

In [7]:
flattened_images = images.reshape(images.shape[0], -1)
print(flattened_images.shape)

(97477, 784)


In [8]:
np.random.seed(42)
num_samples = flattened_images.shape[0]

indices = np.arange(num_samples)
np.random.shuffle(indices)
shuffled_images = flattened_images[indices]
shuffled_labels = labels[indices]

baseline1Idxs = int(num_samples * 0.7)  # 70% for training, 30% for validation
split_index=int(baseline1Idxs*0.7)

train_images = shuffled_images[:split_index]
train_labels = shuffled_labels[:split_index]

val_images = shuffled_images[split_index:]
val_labels = shuffled_labels[split_index:]

In [ ]:
svm=SVMEx()
weights_c1,history_c1=svm.fit(X_train=train_images, y_train=train_labels,
                                X_val=val_images, y_val=val_labels,
                        num_classes=4, epochs=100,
                        learning_rate=0.001, C=0, batch_size=32)

In [7]:
with open ( 'drive/MyDrive/Kaggle2/test_data.pkl' , 'rb' ) as f :
    data = pickle.load ( f )
# Access images and l a b e l s
testimages = data ['images']


In [8]:
testimages=np.array(testimages)
testimages.shape

(1000, 28, 28)

In [ ]:
flattened_test_images = testimages.reshape(testimages.shape[0], -1)

In [ ]:
test_preds=svm.infer(flattened_test_images,weights_c1)

In [ ]:
test_preds==2

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [9]:
def make_original_array(preds):
        """

        """

        index_column = np.arange(1, preds.size + 1)

        # Reshape index_column and labels_train_modified to 2D column vectors
        index_column = index_column.reshape(-1, 1)
        labels_column = preds.reshape(-1, 1)

        # Concatenate the index column with the labels column
        combined_array = np.hstack((index_column, labels_column))

        return combined_array

In [ ]:
predictions=make_original_array(test_preds)

In [ ]:
df = pd.DataFrame(predictions, columns=['ID', 'label'])

# Save the DataFrame to a CSV file
df.to_csv('SVM_baseline1_attempt.csv', index=False)

## Random Forest attempt
Parameters: 100 estimators, random_state=42

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Initialize the RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Fit the model
rf.fit(train_images, train_labels)

# Predict on validation set
val_preds = rf.predict(val_images)

# Calculate accuracy
val_accuracy = accuracy_score(val_labels, val_preds)
print(f'Validation Accuracy: {val_accuracy}')

# Predict on test set
test_preds_rf = rf.predict(flattened_test_images)

# Convert predictions to original array format
predictions_rf = make_original_array(test_preds_rf)

# Create a DataFrame and save to CSV
df_rf = pd.DataFrame(predictions_rf, columns=['ID', 'label'])
df_rf.to_csv('RandomForest_baseline1_attempt.csv', index=False)

## Convolutional neural network attempt
Preprocessing: Reshape images to 4D tensors, reshape labels to categorical variables
CNN: 1st conv2d 32 channels, kernel size 3x3, relu, maxpooling 2x2, 2nd one 64 channels, kernel size 3x3, relu, maxpooling 2x2, flatten, to feed into 2 dense layers: 1st 128, relu, 2nd one 4, softmax for output

In [10]:
# Shuffle the data
np.random.seed(42)
num_samples = images.shape[0]

indices = np.arange(num_samples)
np.random.shuffle(indices)
shuffled_images = images[indices]
shuffled_labels = labels[indices]

# Split into training and validation sets
baseline1Idxs = int(num_samples)  # 80% for training, 20% for validation
split_index = int(baseline1Idxs * 0.8)

train_images = shuffled_images[:split_index]
train_labels = shuffled_labels[:split_index]

val_images = shuffled_images[split_index:]
val_labels = shuffled_labels[split_index:]

# Verify shapes
print(f"Train Images: {train_images.shape}, Train Labels: {train_labels.shape}")
print(f"Validation Images: {val_images.shape}, Validation Labels: {val_labels.shape}")

Train Images: (77981, 28, 28), Train Labels: (77981,)
Validation Images: (19496, 28, 28), Validation Labels: (19496,)


### No Augmentation

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
import torch
from torch.utils.data import DataLoader, TensorDataset

# Reshape images to 4D tensor for CNN input
train_images_cnn = train_images  # Assuming images are 28x28 pixels
val_images_cnn = val_images
#flattened_test_images_cnn = flattened_test_images.reshape(-1, 28, 28, 1)

# Convert labels to categorical
train_labels_cnn = to_categorical(train_labels, num_classes=4)
val_labels_cnn = to_categorical(val_labels, num_classes=4)

# Build the CNN model
cnn_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(4, activation='softmax')
])

# Compile the model
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
cnn_model.fit(train_images_cnn, train_labels_cnn, epochs=10, batch_size=32, validation_data=(val_images_cnn, val_labels_cnn))

# Predict on test set
test_preds_cnn = cnn_model.predict(testimages)
test_preds_cnn_labels = np.argmax(test_preds_cnn, axis=1)

# Convert predictions to original array format
predictions_cnn = make_original_array(test_preds_cnn_labels)

# Create a DataFrame and save to CSV
df_cnn = pd.DataFrame(predictions_cnn, columns=['ID', ' Class'])
df_cnn.to_csv('CNN_baseline2_attempt.csv', index=False)



/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
1950/1950 ━━━━━━━━━━━━━━━━━━━━ 89s 43ms/step - accuracy: 0.6692 - loss: 0.8649 - val_accuracy: 0.7953 - val_loss: 0.5812
Epoch 2/10
1950/1950 ━━━━━━━━━━━━━━━━━━━━ 143s 44ms/step - accuracy: 0.8009 - loss: 0.5637 - val_accuracy: 0.8130 - val_loss: 0.5175
Epoch 3/10
1950/1950 ━━━━━━━━━━━━━━━━━━━━ 134s 40ms/step - accuracy: 0.8265 - loss: 0.4856 - val_accuracy: 0.8322 - val_loss: 0.4698
Epoch 4/10
1950/1950 ━━━━━━━━━━━━━━━━━━━━ 80s 41ms/step - accuracy: 0.8368 - loss: 0.4573 - val_accuracy: 0.8395 - val_loss: 0.4447
Epoch 5/10
1950/1950 ━━━━━━━━━━━━━━━━━━━━ 79s 40ms/step - accuracy: 0.8488 - loss: 0.4249 - val_accuracy: 0.8393 - val_loss: 0.4468
Epoch 6/10
1950/1950 ━━━━━━━━━━━━━━━━━━━━ 82s 40ms/step - accuracy: 0.8555 - loss: 0.3991 - val_accuracy: 0.8458 - val_loss: 0.4323
Epoch 7/10
1950/1950 ━━━━━━━━━━━━━━━━━━━━ 77s 39ms/step - accuracy: 0.8624 - loss: 0.3842 - val_accuracy: 0.8470 - val_loss: 0.4268
Epoch 8/10
1950/1950 ━━━━━━━━━━━━━━━━━━━━ 89s 43ms/step - accuracy: 0.8667

### Augmenting class labels 1 and 2

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import roc_auc_score
import numpy as np

augment_gen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)
# Filter images with labels 1 and 2
mask = np.isin(train_labels, [1, 2])
filtered_images = train_images[mask]
filtered_labels = train_labels[mask]

# Count the number of images with labels 1 and 2
num_filtered_images = len(filtered_images)
print(f"Number of original images with labels 1 and 2: {num_filtered_images}")


# Augment these images
augmented_images = []
augmented_labels = []

for img, lbl in zip(filtered_images, filtered_labels):
    img = np.expand_dims(img, axis=-1)  # Add channel dimension, shape becomes (28, 28, 1)
    img = np.expand_dims(img, axis=0)   # Add batch dimension, shape becomes (1, 28, 28, 1)
    for _ in range(5):  # Generate 5 augmented samples per image
        augmented = next(augment_gen.flow(img, batch_size=1))[0]
        augmented_images.append(augmented)
        augmented_labels.append(lbl)

# Convert augmented data to NumPy arrays
augmented_images = np.array(augmented_images)
augmented_labels = np.array(augmented_labels)

# Number of augmented images
num_augmented_images = len(augmented_images)
print(f"Number of augmented images: {num_augmented_images}")

# Combine augmented data with original training data
train_images_cnn = np.expand_dims(train_images, axis=-1)  # Add channel dimension to training images
train_images_cnn = np.concatenate((train_images_cnn, augmented_images), axis=0)
train_labels_cnn = np.concatenate((train_labels, augmented_labels), axis=0)

# Convert labels to categorical
train_labels_cnn = to_categorical(train_labels_cnn, num_classes=4)
val_images_cnn = np.expand_dims(val_images, axis=-1)  # Add channel dimension to validation images
val_labels_cnn = to_categorical(val_labels, num_classes=4)

# Build the CNN model
cnn_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(4, activation='softmax')
])

# Compile the model
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
cnn_model.fit(train_images_cnn, train_labels_cnn, epochs=10, batch_size=32, validation_data=(val_images_cnn, val_labels_cnn))


# Predict on the validation set
val_preds = cnn_model.predict(val_images_cnn)
val_preds_labels = np.argmax(val_preds, axis=1)

# Calculate Overall AUC
val_labels_flat = np.argmax(val_labels_cnn, axis=1)  # Convert one-hot to class indices
val_auc = roc_auc_score(to_categorical(val_labels_flat, num_classes=4), val_preds, multi_class="ovr")
print(f"Validation AUC: {val_auc:.4f}")




/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
3746/3746 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - accuracy: 0.6455 - loss: 0.8443 - val_accuracy: 0.7960 - val_loss: 0.5552
Epoch 2/10
3746/3746 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - accuracy: 0.8128 - loss: 0.4838 - val_accuracy: 0.8337 - val_loss: 0.4651
Epoch 3/10
3746/3746 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - accuracy: 0.8392 - loss: 0.4172 - val_accuracy: 0.8327 - val_loss: 0.4631
Epoch 4/10
3746/3746 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - accuracy: 0.8550 - loss: 0.3805 - val_accuracy: 0.8250 - val_loss: 0.4992
Epoch 5/10
3746/3746 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - accuracy: 0.8658 - loss: 0.3504 - val_accuracy: 0.8378 - val_loss: 0.4573
Epoch 6/10
3746/3746 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - accuracy: 0.8771 - loss: 0.3220 - val_accuracy: 0.8472 - val_loss: 0.4353
Epoch 7/10
3746/3746 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - accuracy: 0.8870 - loss: 0.2994 - val_accuracy: 0.8483 - val_loss: 0.4248
Epoch 8/10
3746/3746 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - accuracy: 0.8936 - loss: 0.2787 -

In [ ]:
# Predict on test set
test_images_cnn= np.expand_dims(testimages, axis=-1)
test_preds_cnn = cnn_model.predict(test_images_cnn)
test_preds_cnn_labels = np.argmax(test_preds_cnn, axis=1)

# Convert predictions to original array format
predictions_cnn = make_original_array(test_preds_cnn_labels)

# Create a DataFrame and save to CSV
df_cnn = pd.DataFrame(predictions_cnn, columns=['ID', ' Class'])
df_cnn.to_csv('CNN_baseline_aug_attempt.csv', index=False)


32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step


In [ ]:
df_cnn

,ID,Class
0,1,3
1,2,0
2,3,3
3,4,3
4,5,0
...,...,...
995,996,2
996,997,1
997,998,3
998,999,3


In [ ]:
cnn_no_aug=pd.read_csv('CNN_baseline2_attempt.csv')
cnn_no_aug

,ID,Class
0,1,3
1,2,0
2,3,3
3,4,3
4,5,0
...,...,...
995,996,0
996,997,1
997,998,3
998,999,3


In [ ]:
# Assuming the predicted classes are in a column named 'Class'
differences = (df_cnn[' Class'] != cnn_no_aug[' Class']).sum()
total = len(df_cnn)

print(f"Total Predictions: {total}")
print(f"Different Predictions: {differences}")
print(f"Percentage Difference: {(differences / total) * 100:.2f}%")


Total Predictions: 1000
Different Predictions: 203
Percentage Difference: 20.30%


### More augmentation for class 1 and 2 and a little for class 0 and 3

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import roc_auc_score
import numpy as np

augment_gen = ImageDataGenerator(
    rotation_range=30,          # Increase rotation
    width_shift_range=0.2,      # Increase horizontal shift
    height_shift_range=0.2,     # Increase vertical shift
    zoom_range=0.2,             # Add zoom augmentation
    shear_range=15,             # Add shearing
    horizontal_flip=True,       # Allow horizontal flips
    fill_mode='nearest'         # Fill missing pixels
)

# Filter images with labels 1 and 2
mask1 = np.isin(train_labels, [1, 2])
filtered_images = train_images[mask1]
filtered_labels = train_labels[mask1]

# Count the number of images with labels 1 and 2
num_filtered_images = len(filtered_images)
print(f"Number of original images with labels 1 and 2: {num_filtered_images}")


# Augment these images
augmented_images = []
augmented_labels = []

for img, lbl in zip(filtered_images, filtered_labels):
    img = np.expand_dims(img, axis=-1)  # Add channel dimension, shape becomes (28, 28, 1)
    img = np.expand_dims(img, axis=0)   # Add batch dimension, shape becomes (1, 28, 28, 1)
    for _ in range(10):  # Generate 5 augmented samples per image
        augmented = next(augment_gen.flow(img, batch_size=1))[0]
        augmented_images.append(augmented)
        augmented_labels.append(lbl)

mask2 = np.isin(train_labels, [0, 3])
filtered_images = train_images[mask2]
filtered_labels = train_labels[mask2]

# Count the number of images with labels 1 and 2
num_filtered_images = len(filtered_images)
print(f"Number of original images with labels 0 and 3: {num_filtered_images}")


for img, lbl in zip(filtered_images, filtered_labels):
    img = np.expand_dims(img, axis=-1)  # Add channel dimension, shape becomes (28, 28, 1)
    img = np.expand_dims(img, axis=0)   # Add batch dimension, shape becomes (1, 28, 28, 1)
    for _ in range(2):  # Generate 5 augmented samples per image
        augmented = next(augment_gen.flow(img, batch_size=1))[0]
        augmented_images.append(augmented)
        augmented_labels.append(lbl)


# Convert augmented data to NumPy arrays
augmented_images = np.array(augmented_images)
augmented_labels = np.array(augmented_labels)

# Number of augmented images
num_augmented_images = len(augmented_images)
print(f"Number of augmented images: {num_augmented_images}")

# Combine augmented data with original training data
train_images_cnn = np.expand_dims(train_images, axis=-1)  # Add channel dimension to training images
train_images_cnn = np.concatenate((train_images_cnn, augmented_images), axis=0)
train_labels_cnn = np.concatenate((train_labels, augmented_labels), axis=0)

# Convert labels to categorical
train_labels_cnn = to_categorical(train_labels_cnn, num_classes=4)
val_images_cnn = np.expand_dims(val_images, axis=-1)  # Add channel dimension to validation images
val_labels_cnn = to_categorical(val_labels, num_classes=4)

# Build the CNN model
cnn_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(4, activation='softmax')
])

# Compile the model
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
cnn_model.fit(train_images_cnn, train_labels_cnn, epochs=15, batch_size=32, validation_data=(val_images_cnn, val_labels_cnn))


# Predict on the validation set
val_preds = cnn_model.predict(val_images_cnn)
val_preds_labels = np.argmax(val_preds, axis=1)

# Calculate Overall AUC
val_labels_flat = np.argmax(val_labels_cnn, axis=1)  # Convert one-hot to class indices
val_auc = roc_auc_score(to_categorical(val_labels_flat, num_classes=4), val_preds, multi_class="ovr")
print(f"Validation AUC: {val_auc:.4f}")



Number of original images with labels 1 and 2: 11497
Number of original images with labels 0 and 3: 50887
Number of augmented images: 216744


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/15
8723/8723 ━━━━━━━━━━━━━━━━━━━━ 25s 3ms/step - accuracy: 0.5611 - loss: 1.0147 - val_accuracy: 0.8044 - val_loss: 0.5264
Epoch 2/15
8723/8723 ━━━━━━━━━━━━━━━━━━━━ 19s 2ms/step - accuracy: 0.6925 - loss: 0.7568 - val_accuracy: 0.8300 - val_loss: 0.4737
Epoch 3/15
8723/8723 ━━━━━━━━━━━━━━━━━━━━ 19s 2ms/step - accuracy: 0.7211 - loss: 0.6916 - val_accuracy: 0.8354 - val_loss: 0.4570
Epoch 4/15
8723/8723 ━━━━━━━━━━━━━━━━━━━━ 19s 2ms/step - accuracy: 0.7395 - loss: 0.6488 - val_accuracy: 0.8428 - val_loss: 0.4418
Epoch 5/15
8723/8723 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step - accuracy: 0.7518 - loss: 0.6189 - val_accuracy: 0.8491 - val_loss: 0.4207
Epoch 6/15
8723/8723 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step - accuracy: 0.7640 - loss: 0.5931 - val_accuracy: 0.8481 - val_loss: 0.4210
Epoch 7/15
8723/8723 ━━━━━━━━━━━━━━━━━━━━ 19s 2ms/step - accuracy: 0.7706 - loss: 0.5727 - val_accuracy: 0.8522 - val_loss: 0.4256
Epoch 8/15
8723/8723 ━━━━━━━━━━━━━━━━━━━━ 19s 2ms/step - accuracy: 0.7799 - loss: 0

In [ ]:
# Predict on test set
test_images_cnn= np.expand_dims(testimages, axis=-1)
test_preds_cnn = cnn_model.predict(test_images_cnn)
test_preds_cnn_labels = np.argmax(test_preds_cnn, axis=1)

# Convert predictions to original array format
predictions_cnn = make_original_array(test_preds_cnn_labels)

# Create a DataFrame and save to CSV
df_cnn_augb = pd.DataFrame(predictions_cnn, columns=['ID', ' Class'])
df_cnn_augb.to_csv('CNN_baseline_augboth_attempt.csv', index=False)

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


In [ ]:
# Assuming the predicted classes are in a column named 'Class'
differences = (df_cnn[' Class'] != df_cnn_augb[' Class']).sum()
total = len(df_cnn)

print(f"Total Predictions: {total}")
print(f"Different Predictions: {differences}")
print(f"Percentage Difference: {(differences / total) * 100:.2f}%")

Total Predictions: 1000
Different Predictions: 221
Percentage Difference: 22.10%


**Same model using Pytorch**

In [ ]:
from sklearn.metrics import roc_auc_score

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Reshape images to 4D tensor for CNN input
train_images_cnn = train_images.reshape(-1, 1, 28, 28)  # Assuming images are 28x28 pixels
val_images_cnn = val_images.reshape(-1, 1, 28, 28)
flattened_test_images_cnn = flattened_test_images.reshape(-1, 1, 28, 28)

# Convert labels to categorical
train_labels_cnn = torch.tensor(train_labels, dtype=torch.long)
val_labels_cnn = torch.tensor(val_labels, dtype=torch.long)

# Convert data to PyTorch tensors
train_images_tensor = torch.tensor(train_images_cnn, dtype=torch.float32).to(device)
train_labels_tensor = torch.tensor(train_labels, dtype=torch.long).to(device)
val_images_tensor = torch.tensor(val_images_cnn, dtype=torch.float32).to(device)
val_labels_tensor = torch.tensor(val_labels, dtype=torch.long).to(device)
test_images_tensor = torch.tensor(flattened_test_images_cnn, dtype=torch.float32).to(device)

# Create DataLoader
train_dataset = TensorDataset(train_images_tensor, train_labels_tensor)
val_dataset = TensorDataset(val_images_tensor, val_labels_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Define the CNN model
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(64 * 5 * 5, 128)
        self.fc2 = nn.Linear(128, 4)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize the model, loss function, and optimizer
cnn_model = CNNModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn_model.parameters(), lr=0.001)

# Train the model
num_epochs = 10
val_accuracies = []
val_losses = []
for epoch in range(num_epochs):
    cnn_model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = cnn_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Validate the model
    cnn_model.eval()
    val_loss = 0
    val_correct = 0
    all_labels = []
    all_preds = []
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = cnn_model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            val_correct += (predicted == labels).sum().item()
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(F.softmax(outputs, dim=1).cpu().numpy())

    val_loss /= len(val_loader)
    val_accuracy = val_correct / len(val_dataset)
    val_accuracies.append(val_accuracy)
    val_losses.append(val_loss)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {val_loss}, Accuracy: {val_accuracy}')

# Save the model state
torch.save(cnn_model.state_dict(), 'cnn_model.pth')

# Compute AUC for each class
all_labels = np.array(all_labels)
all_preds = np.array(all_preds)
auc_scores = []
for i in range(4):
    auc = roc_auc_score(all_labels == i, all_preds[:, i])
    auc_scores.append(auc)
    print(f'AUC for class {i}: {auc}')

# Predict on test set
cnn_model.eval()
with torch.no_grad():
    test_outputs = cnn_model(test_images_tensor)
    _, test_preds_cnn_labels = torch.max(test_outputs, 1)

# Convert predictions to original array format
predictions_cnn = make_original_array(test_preds_cnn_labels.cpu().numpy())

# Create a DataFrame and save to CSV
df_cnn = pd.DataFrame(predictions_cnn, columns=['ID', 'label'])
df_cnn.to_csv('CNN_baseline1_attempt.csv', index=False)

Epoch 1/10, Loss: 0.6258089580936321, Accuracy: 0.7795993080420003
Epoch 2/10, Loss: 0.5465488908872335, Accuracy: 0.8069759021603572
Epoch 3/10, Loss: 0.5201600855036592, Accuracy: 0.8150018103552319
Epoch 4/10, Loss: 0.4983069887636481, Accuracy: 0.822927143259444
Epoch 5/10, Loss: 0.4930240340252794, Accuracy: 0.8230277185501066
Epoch 6/10, Loss: 0.47727027386810483, Accuracy: 0.8315162730820292
Epoch 7/10, Loss: 0.4605699416676697, Accuracy: 0.836826648429014
Epoch 8/10, Loss: 0.45732239249283435, Accuracy: 0.836283541859436
Epoch 9/10, Loss: 0.4551902298924622, Accuracy: 0.8389990747073259
Epoch 10/10, Loss: 0.44121442443692394, Accuracy: 0.8434646176127449
AUC for class 0: 0.9798477641800922
AUC for class 1: 0.9469241140223681
AUC for class 2: 0.8418766190799161
AUC for class 3: 0.9601584625074469


# This cell defines and compiles a ResNet50 model using TensorFlow and Keras.
# The model is designed for image classification tasks and consists of the following components:
#
# 1. **Base Model**:
#    - Architecture: ResNet50
#    - Pre-trained Weights: ImageNet
#    - Include Top: False (excluding the top fully connected layer)
#    - Input Shape: (28, 28, 3) (assuming input images are 28x28 pixels with 3 color channels)
#
# 2. **Custom Layers**:
#    - GlobalAveragePooling2D: Reduces each feature map to a single value.
#    - Dense Layer:
#      - Units: 128
#      - Activation: ReLU
#    - Dropout Layer:
#      - Rate: 0.5 (to prevent overfitting)
#    - Dense Layer:
#      - Units: 4 (assuming 4 classes for classification)
#      - Activation: Softmax
#
# The model is compiled with the following parameters:
# - **Optimizer**: Adam
# - **Loss Function**: Categorical Crossentropy (suitable for multi-class classification)
# - **Metrics**: Accuracy (to monitor the accuracy during training and evaluation)
#
# Design Choices:
# - The use of ResNet50 as the base model leverages pre-trained weights to improve feature extraction.
# - GlobalAveragePooling2D is used to reduce the spatial dimensions of the feature maps.
# - ReLU activation is chosen for its effectiveness in introducing non-linearity.
# - Dropout is applied to prevent overfitting.
# - The final Dense layer uses Softmax activation to output a probability distribution over the classes.
# - Adam optimizer is selected for its efficiency and adaptive learning rate capabilities.
# - Categorical Crossentropy is used as the loss function to handle one-hot encoded labels.


In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model

# Load the ResNet50 model with pre-trained weights, excluding the top layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(28, 28, 3))

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(4, activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Convert grayscale images to 3 channels by repeating the single channel 3 times
train_images_resnet = np.repeat(train_images_cnn, 3, axis=-1)
val_images_resnet = np.repeat(val_images_cnn, 3, axis=-1)
flattened_test_images_resnet = np.repeat(flattened_test_images_cnn, 3, axis=-1)

# Train the model
model.fit(train_images_resnet, train_labels_cnn, epochs=10, batch_size=32, validation_data=(val_images_resnet, val_labels_cnn))

# Predict on test set
test_preds_resnet = model.predict(flattened_test_images_resnet)
test_preds_resnet_labels = np.argmax(test_preds_resnet, axis=1)

# Convert predictions to original array format
predictions_resnet = make_original_array(test_preds_resnet_labels)

# Create a DataFrame and save to CSV
df_resnet = pd.DataFrame(predictions_resnet, columns=['ID', 'label'])
df_resnet.to_csv('ResNet_baseline1_attempt.csv', index=False)


## ResNet50 attempt using PyTorch
Parameters: 100 epochs, learning rate=0.001, batch size=32, dropout rate=0.5

Preprocessing: Convert grayscale images to 3 channels by repeating the single channel 3 times

Model Architecture:
- Base Model: ResNet50 with pre-trained weights (ImageNet)
- Custom Layers:
    - Global Average Pooling
    - Dense Layer: 128 units, ReLU activation
    - Dropout Layer: 0.5 rate
    - Dense Layer: 4 units, Softmax activation

Training:
- Loss Function: CrossEntropyLoss
- Optimizer: Adam

Evaluation:
- Validation Accuracy and Loss per epoch
- Predictions on test set saved to CSV file

In [ ]:
import torchvision.models as models

# Load the ResNet50 model with pre-trained weights for retina medical images
base_model = models.resnet50(weights='IMAGENET1K_V2').to(device)
base_model.fc = nn.Identity()  # Remove the final fully connected layer

# Add custom layers on top of the base model
class CustomResNet(nn.Module):
    def __init__(self, base_model):
        super(CustomResNet, self).__init__()
        self.base_model = base_model
        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc1 = nn.Linear(base_model.fc.in_features, 128)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 4)

    def forward(self, x):
        x = self.base_model(x)
        x = self.global_avg_pool(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

model = CustomResNet(base_model).to(device)

# Freeze the layers of the base model
for param in base_model.parameters():
    param.requires_grad = False

# Convert grayscale images to 3 channels by repeating the single channel 3 times
train_images_resnet = np.repeat(train_images_cnn, 3, axis=1)
val_images_resnet = np.repeat(val_images_cnn, 3, axis=1)
flattened_test_images_resnet = np.repeat(flattened_test_images_cnn, 3, axis=1)

# Convert data to PyTorch tensors
train_images_tensor = torch.tensor(train_images_resnet, dtype=torch.float32).to(device)
train_labels_tensor = torch.tensor(train_labels, dtype=torch.long).to(device)
val_images_tensor = torch.tensor(val_images_resnet, dtype=torch.float32).to(device)
val_labels_tensor = torch.tensor(val_labels, dtype=torch.long).to(device)
test_images_tensor = torch.tensor(flattened_test_images_resnet, dtype=torch.float32).to(device)

# Create DataLoader
train_dataset = TensorDataset(train_images_tensor, train_labels_tensor)
val_dataset = TensorDataset(val_images_tensor, val_labels_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Initialize the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 10
val_accuracies = []
val_losses = []
for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Validate the model
    model.eval()
    val_loss = 0
    val_correct = 0
    all_labels = []
    all_preds = []
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            val_correct += (predicted == labels).sum().item()
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(F.softmax(outputs, dim=1).cpu().numpy())

    val_loss /= len(val_loader)
    val_accuracy = val_correct / len(val_dataset)
    val_accuracies.append(val_accuracy)
    val_losses.append(val_loss)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {val_loss}, Accuracy: {val_accuracy}')

# Save the model state
torch.save(model.state_dict(), 'resnet_model.pth')

# Compute AUC for each class
all_labels = np.array(all_labels)
all_preds = np.array(all_preds)
auc_scores = []
for i in range(4):
    auc = roc_auc_score(all_labels == i, all_preds[:, i])
    auc_scores.append(auc)
    print(f'AUC for class {i}: {auc}')

# Predict on test set
model.eval()
with torch.no_grad():
    test_outputs = model(test_images_tensor)
    _, test_preds_resnet_labels = torch.max(test_outputs, 1)

# Convert predictions to original array format
predictions_resnet = make_original_array(test_preds_resnet_labels.cpu().numpy())

# Create a DataFrame and save to CSV
df_resnet = pd.DataFrame(predictions_resnet, columns=['ID', 'label'])
df_resnet.to_csv('ResNet_baseline1_attempt.csv', index=False)

### Trying out vision transfomers and performing augmentations

In [ ]:
!pip install --upgrade pip



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 19.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [ ]:
from sklearn.metrics import roc_auc_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import timm
import numpy as np

# Custom Dataset with Augmentation
class CustomDataset(Dataset):
    def __init__(self, images, labels, transform=None, augment_classes=None):
        self.images = images
        self.labels = labels
        self.transform = transform
        self.augment_classes = augment_classes
        self.augmented_images = []
        self.augmented_labels = []

        # Perform augmentations for specified classes
        if self.augment_classes:
            for i, label in enumerate(labels):
                if label in self.augment_classes:
                    augmented_image = self.apply_augmentation(images[i])
                    self.augmented_images.append(augmented_image)
                    self.augmented_labels.append(label)
            # Convert augmented images to the same shape as original images
            self.augmented_images = np.array(self.augmented_images).squeeze()  # Remove extra channel if added
            self.images = np.concatenate((self.images, self.augmented_images), axis=0)
            self.labels = np.concatenate((self.labels, np.array(self.augmented_labels)), axis=0)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

    def apply_augmentation(self, image):
        """Apply augmentations and return an image with the same dimensions."""
        aug_transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(15),
            transforms.ColorJitter(brightness=0.2, contrast=0.2),
            transforms.ToTensor()
        ])
        augmented_image = aug_transform(image)
        return augmented_image.squeeze(0).numpy()  # Convert back to (height, width)

transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Grayscale(num_output_channels=3),  # Convert grayscale to 3 channels
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize for 3 channels
])

# Dataset Preparation
dataset = CustomDataset(images=images, labels=labels, transform=transform, augment_classes=[1, 2])
dataset_size = len(dataset)
train_size = int(0.8 * dataset_size)
val_size = dataset_size - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Vision Transformer Model
class ViTModel28x28(nn.Module):
    def __init__(self, num_classes=4):
        super(ViTModel28x28, self).__init__()
        # Use vit_base_patch16_224 and adjust img_size
        self.vit = timm.create_model(
            'vit_base_patch16_224',
            pretrained=True,
            img_size=28,  # Input size
            patch_size=7  # Adjust patch size for 28x28 images
        )
        self.vit.head = nn.Linear(self.vit.head.in_features, num_classes)  # Replace the head

    def forward(self, x):
        return self.vit(x)

# Initialize Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ViTModel28x28(num_classes=4).to(device)


# Initialize Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ViTModel28x28(num_classes=4).to(device)

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Function to Calculate AUC
def calculate_auc(loader, model, num_classes=4):
    model.eval()
    all_labels = []
    all_outputs = []

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            probabilities = nn.Softmax(dim=1)(outputs)

            all_labels.append(labels.cpu().numpy())
            all_outputs.append(probabilities.cpu().numpy())

    # Convert to NumPy arrays
    all_labels = np.concatenate(all_labels)
    all_outputs = np.concatenate(all_outputs)

    # One-hot encode labels
    one_hot_labels = np.eye(num_classes)[all_labels]

    # Calculate AUC
    auc = roc_auc_score(one_hot_labels, all_outputs, multi_class="ovr")
    return auc

# Training Loop
def train_vit(model, train_loader, val_loader, epochs=10):
    for epoch in range(epochs):
        # Training Phase
        model.train()
        train_loss = 0
        train_correct = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward Pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            train_loss += loss.item()

            # Backward Pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Accuracy
            _, preds = torch.max(outputs, 1)
            train_correct += (preds == labels).sum().item()

        # Validation Phase
        model.eval()
        val_correct = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, preds = torch.max(outputs, 1)
                val_correct += (preds == labels).sum().item()

        # Calculate metrics
        train_accuracy = train_correct / len(train_loader.dataset)
        val_accuracy = val_correct / len(val_loader.dataset)
        val_auc = calculate_auc(val_loader, model)

        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, "
              f"Val Accuracy: {val_accuracy:.4f}, Val AUC: {val_auc:.4f}")

# Train the Model
train_vit(model, train_loader, val_loader, epochs=10)


Epoch 1/10, Train Loss: 3637.1604, Train Accuracy: 0.4629, Val Accuracy: 0.5231, Val AUC: 0.6864
Epoch 2/10, Train Loss: 2980.7136, Train Accuracy: 0.5943, Val Accuracy: 0.6387, Val AUC: 0.8030
Epoch 3/10, Train Loss: 2597.0436, Train Accuracy: 0.6579, Val Accuracy: 0.6701, Val AUC: 0.8355
Epoch 4/10, Train Loss: 2443.5494, Train Accuracy: 0.6780, Val Accuracy: 0.6436, Val AUC: 0.8135
Epoch 5/10, Train Loss: 2603.2740, Train Accuracy: 0.6545, Val Accuracy: 0.6837, Val AUC: 0.8417
Epoch 6/10, Train Loss: 2613.0445, Train Accuracy: 0.6512, Val Accuracy: 0.6843, Val AUC: 0.8527
Epoch 7/10, Train Loss: 2448.0301, Train Accuracy: 0.6743, Val Accuracy: 0.6924, Val AUC: 0.8577
Epoch 8/10, Train Loss: 2422.3131, Train Accuracy: 0.6776, Val Accuracy: 0.6717, Val AUC: 0.8388


KeyboardInterrupt: 

### ResNet-50 from scratch

In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from sklearn.metrics import roc_auc_score
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Custom Dataset
class CustomDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

augment_gen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    shear_range=15,
    horizontal_flip=True,
    fill_mode="nearest"
)

# Augment images with labels 1 and 2
mask1 = np.isin(train_labels, [1, 2])
filtered_images_1_2 = train_images[mask1]
filtered_labels_1_2 = train_labels[mask1]

augmented_images = []
augmented_labels = []

for img, lbl in zip(filtered_images_1_2, filtered_labels_1_2):
    img = np.expand_dims(img, axis=-1)  # Add channel dimension, shape becomes (28, 28, 1)
    img = np.expand_dims(img, axis=0)   # Add batch dimension, shape becomes (1, 28, 28, 1)
    for _ in range(10):  # Generate 10 augmented samples per image
        augmented = next(augment_gen.flow(img, batch_size=1))[0]
        augmented_images.append(augmented)
        augmented_labels.append(lbl)

# Augment images with labels 0 and 3
mask2 = np.isin(train_labels, [0, 3])
filtered_images_0_3 = train_images[mask2]
filtered_labels_0_3 = train_labels[mask2]

for img, lbl in zip(filtered_images_0_3, filtered_labels_0_3):
    img = np.expand_dims(img, axis=-1)  # Add channel dimension, shape becomes (28, 28, 1)
    img = np.expand_dims(img, axis=0)   # Add batch dimension, shape becomes (1, 28, 28, 1)
    for _ in range(2):  # Generate 2 augmented samples per image
        augmented = next(augment_gen.flow(img, batch_size=1))[0]
        augmented_images.append(augmented)
        augmented_labels.append(lbl)

# Convert augmented data to NumPy arrays
augmented_images = np.array(augmented_images)
augmented_labels = np.array(augmented_labels)

# Combine augmented data with original training data
train_images_cnn = np.expand_dims(train_images, axis=-1)  # Add channel dimension to training images
train_images_cnn = np.concatenate((train_images_cnn, augmented_images), axis=0)
train_labels_cnn = np.concatenate((train_labels, augmented_labels), axis=0)

# Convert labels to categorical

val_images_cnn = np.expand_dims(val_images, axis=-1)  # Add channel dimension to validation images
train_labels_cnn = np.argmax(to_categorical(train_labels_cnn, num_classes=4), axis=1)
val_labels_cnn = np.argmax(to_categorical(val_labels, num_classes=4), axis=1)


# PyTorch Dataset and DataLoader
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize
])

train_dataset = CustomDataset(train_images_cnn, train_labels_cnn, transform=transform)
val_dataset = CustomDataset(val_images, val_labels, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Bottleneck Block for ResNet-50
class Bottleneck(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(out_channels, out_channels * 4, kernel_size=1, stride=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample

    def forward(self, x):
        identity = x
        if self.downsample is not None:
            identity = self.downsample(x)

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.conv3(out)
        out = self.bn3(out)

        out += identity
        out = self.relu(out)
        return out

# ResNet-50 for 28x28 Inputs
class ResNet28x28(nn.Module):
    def __init__(self, block, layers, num_classes=4):
        super(ResNet28x28, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=False)  # Adjusted for 28x28
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=1)  # No stride for 28x28 input
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * 4, num_classes)

    def _make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if stride != 1 or self.in_channels != out_channels * 4:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, out_channels * 4, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * 4),
            )

        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels * 4
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, out_channels))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

# Training Loop with AUC Calculation
def train_resnet28(model, train_loader, val_loader, epochs=10):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(epochs):
        model.train()
        train_loss = 0
        train_correct = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            train_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            _, preds = torch.max(outputs, 1)
            train_correct += (preds == labels).sum().item()

        # Validation
        model.eval()
        val_correct = 0
        all_preds = []
        all_labels = []


        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                _, preds = torch.max(outputs, 1)
                val_correct += (preds == labels).sum().item()

                # Store predicted probabilities and true class indices
                all_preds.extend(torch.softmax(outputs, dim=1).cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        # Calculate AUC
        val_auc = roc_auc_score(
            np.eye(4)[np.array(all_labels)],  # Convert class indices to one-hot encoded for AUC
            np.array(all_preds),
            multi_class='ovr'
        )


        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, "
              f"Train Accuracy: {train_correct / len(train_loader.dataset):.4f}, "
              f"Val Accuracy: {val_correct / len(val_loader.dataset):.4f}, Val AUC: {val_auc:.4f}")

# Instantiate Model and Train
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ResNet28x28(Bottleneck, [3, 4, 6, 3], num_classes=4).to(device)

train_resnet28(model, train_loader, val_loader, epochs=15)


Epoch 1/15, Train Loss: 6419.6298, Train Accuracy: 0.7022, Val Loss: 467.4983, Val Accuracy: 0.8462, Val AUC: 0.9455


KeyboardInterrupt: 

## BEST ONE SO FAR

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from sklearn.metrics import roc_auc_score
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Custom Dataset
class CustomDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label


# Data Augmentation
augment_gen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    shear_range=15,
    horizontal_flip=True,
    fill_mode="nearest"
)

# Augment images with labels 1 and 2
mask1 = np.isin(train_labels, [1, 2])
filtered_images_1_2 = train_images[mask1]
filtered_labels_1_2 = train_labels[mask1]

augmented_images = []
augmented_labels = []

for img, lbl in zip(filtered_images_1_2, filtered_labels_1_2):
    img = np.expand_dims(img, axis=-1)  # Add channel dimension
    img = np.expand_dims(img, axis=0)   # Add batch dimension
    for _ in range(10):  # Generate 10 augmented samples per image
        augmented = next(augment_gen.flow(img, batch_size=1))[0]
        augmented_images.append(augmented)
        augmented_labels.append(lbl)

# Augment images with labels 0 and 3
mask2 = np.isin(train_labels, [0, 3])
filtered_images_0_3 = train_images[mask2]
filtered_labels_0_3 = train_labels[mask2]

for img, lbl in zip(filtered_images_0_3, filtered_labels_0_3):
    img = np.expand_dims(img, axis=-1)  # Add channel dimension
    img = np.expand_dims(img, axis=0)   # Add batch dimension
    for _ in range(2):  # Generate 2 augmented samples per image
        augmented = next(augment_gen.flow(img, batch_size=1))[0]
        augmented_images.append(augmented)
        augmented_labels.append(lbl)

# Combine augmented data with original training data
augmented_images = np.array(augmented_images)
augmented_labels = np.array(augmented_labels)
train_images_cnn = np.expand_dims(train_images, axis=-1)  # Add channel dimension to training images
train_images_cnn = np.concatenate((train_images_cnn, augmented_images), axis=0)
train_labels_cnn = np.concatenate((train_labels, augmented_labels), axis=0)

# Convert labels to class indices
train_labels_cnn = train_labels_cnn.astype(int)
val_labels = val_labels.astype(int)

# PyTorch Dataset and DataLoader
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize
])

train_dataset = CustomDataset(train_images_cnn, train_labels_cnn, transform=transform)
val_dataset = CustomDataset(val_images, val_labels, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Bottleneck Block for ResNet-50
class Bottleneck(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(out_channels, out_channels * 4, kernel_size=1, stride=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample

    def forward(self, x):
        identity = x
        if self.downsample is not None:
            identity = self.downsample(x)

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.conv3(out)
        out = self.bn3(out)

        out += identity
        out = self.relu(out)
        return out


# ResNet-50 for 28x28 Inputs
class ResNet28x28(nn.Module):
    def __init__(self, block, layers, num_classes=4):
        super(ResNet28x28, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=False)  # Adjusted for 28x28
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=1)  # No stride for 28x28 input
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * 4, num_classes)

    def _make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if stride != 1 or self.in_channels != out_channels * 4:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, out_channels * 4, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * 4),
            )

        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels * 4
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, out_channels))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x


# Compute Class Weights for Balanced Loss
class_weights = compute_class_weight('balanced', classes=np.unique(train_labels_cnn), y=train_labels_cnn)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)


# Temperature Scaled Cross Entropy Loss
class TemperatureScaledCrossEntropyLoss(nn.Module):
    def __init__(self, temperature=2.0):
        super(TemperatureScaledCrossEntropyLoss, self).__init__()
        self.temperature = temperature
        self.criterion = nn.CrossEntropyLoss(weight=class_weights)

    def forward(self, logits, targets):
        scaled_logits = logits / self.temperature
        return self.criterion(scaled_logits, targets)

# Training Loop with AUC Calculation and Dynamic LR Adjustment
# Training Loop with AUC Calculation and Dynamic LR Adjustment
def train_resnet28(model, train_loader, val_loader, epochs=15):
    # Optimizer and Criterion
    criterion = TemperatureScaledCrossEntropyLoss(temperature=2.0)
    optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)

    # Learning Rate Scheduler Placeholder
    scheduler_active = False  # To ensure we reduce LR only after crossing accuracy
    reduced_lr = 1e-5  # New learning rate after crossing 85% accuracy
    reduced_lr_2=3e-6
    reduced_lr_3=1e-6
    for epoch in range(epochs):
        # Training Phase
        model.train()
        train_loss = 0
        train_correct = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)  # Ensure tensors are on the correct device
            outputs = model(images)
            loss = criterion(outputs, labels)
            train_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Gradient clipping
            optimizer.step()

            _, preds = torch.max(outputs, 1)
            train_correct += (preds == labels).sum().item()

        # Validation Phase
        model.eval()
        val_loss = 0
        val_correct = 0
        all_preds = []
        all_labels = []

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)  # Explicitly move to device
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                _, preds = torch.max(outputs, 1)
                val_correct += (preds == labels).sum().item()

                # Store predicted probabilities and true class indices
                all_preds.extend(torch.softmax(outputs, dim=1).cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        # Calculate AUC
        val_auc = roc_auc_score(
            np.eye(4)[np.array(all_labels)],  # Convert class indices to one-hot encoded for AUC
            np.array(all_preds),
            multi_class='ovr'
        )

        # Metrics
        train_accuracy = train_correct / len(train_loader.dataset)
        val_accuracy = val_correct / len(val_loader.dataset)
        new_lr = None
        if train_accuracy > 0.91:
            new_lr = reduced_lr_3
            print(f"Learning rate reduced to {new_lr:.1e} after epoch {epoch+1}")
        elif train_accuracy > 0.88:
            new_lr = reduced_lr_2
            print(f"Learning rate reduced to {new_lr:.1e} after epoch {epoch+1}")
        elif train_accuracy > 0.85 and not scheduler_active:
            new_lr = reduced_lr
            scheduler_active = True  # Only adjust once for this threshold
            print(f"Learning rate reduced to {new_lr:.1e} after epoch {epoch+1}")
        else:
            new_lr = None

        # Update learning rate if changed
        if new_lr is not None:
            for param_group in optimizer.param_groups:
                param_group['lr'] = new_lr

        # Print epoch metrics
        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, "
              f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}, Val AUC: {val_auc:.4f}")

# Instantiate Model and Train
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
assert torch.cuda.is_available(), "CUDA is not available. Ensure GPU drivers are installed and CUDA is supported."
model = ResNet28x28(Bottleneck, [3, 4, 6, 3], num_classes=4).to(device)
assert next(model.parameters()).is_cuda, "Model is not on GPU!"
train_resnet28(model, train_loader, val_loader, epochs=15)




Using device: cuda
Epoch 1/15, Train Loss: 15093.3895, Train Accuracy: 0.7248, Val Loss: 508.8144, Val Accuracy: 0.8400, Val AUC: 0.9513
Epoch 2/15, Train Loss: 11715.1383, Train Accuracy: 0.7913, Val Loss: 447.9826, Val Accuracy: 0.8827, Val AUC: 0.9612
Epoch 3/15, Train Loss: 10592.5576, Train Accuracy: 0.8137, Val Loss: 415.3051, Val Accuracy: 0.8857, Val AUC: 0.9653
Epoch 4/15, Train Loss: 9849.7282, Train Accuracy: 0.8278, Val Loss: 427.4391, Val Accuracy: 0.8730, Val AUC: 0.9636
Epoch 5/15, Train Loss: 9236.1505, Train Accuracy: 0.8384, Val Loss: 384.7570, Val Accuracy: 0.8937, Val AUC: 0.9687
Epoch 6/15, Train Loss: 8742.9923, Train Accuracy: 0.8481, Val Loss: 377.3969, Val Accuracy: 0.8968, Val AUC: 0.9700
Learning rate reduced to 1.0e-05 after epoch 7
Epoch 7/15, Train Loss: 8247.8865, Train Accuracy: 0.8576, Val Loss: 377.4650, Val Accuracy: 0.9003, Val AUC: 0.9715
Learning rate reduced to 3.0e-06 after epoch 8
Epoch 8/15, Train Loss: 6123.6544, Train Accuracy: 0.8964, Val Lo

In [12]:
print(f"Number of augmented images: {augmented_images.shape[0]}")

Number of augmented images: 270802


In [ ]:
test_images_cnn= np.expand_dims(testimages, axis=-1)
test_preds_cnn = cnn_model.predict(test_images_cnn)
test_preds_cnn_labels = np.argmax(test_preds_cnn, axis=1)

# Convert predictions to original array format
predictions_cnn = make_original_array(test_preds_cnn_labels)

# Create a DataFrame and save to CSV
df_cnn_augb = pd.DataFrame(predictions_cnn, columns=['ID', ' Class'])
df_cnn_augb.to_csv('CNN_baseline_augboth_attempt.csv', index=False)

In [13]:
import torch
import numpy as np
from torchvision import transforms

# Assuming testimages is a NumPy array of shape (num_samples, 28, 28)

# Preprocess test images
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # Ensure same normalization as during training
])

# Convert testimages to PyTorch Tensor with proper dimensions
test_images_tensor = torch.stack([test_transform(img) for img in testimages])  # Shape: (num_samples, 1, 28, 28)

# Send to device (e.g., GPU or CPU)
test_images_tensor = test_images_tensor.to(device)

# Ensure the model is in evaluation mode
model.eval()

# Make predictions
with torch.no_grad():
    logits = model(test_images_tensor)  # Shape: (num_samples, num_classes)
    probabilities = torch.softmax(logits, dim=1)  # Convert logits to probabilities
    predicted_classes = torch.argmax(probabilities, dim=1)  # Get class with highest probability

# Convert predictions to NumPy array
predicted_classes_np = predicted_classes.cpu().numpy()

# Print predictions
print("Predicted Classes:", predicted_classes_np)


Predicted Classes: [3 0 3 3 0 2 1 0 0 3 0 1 3 3 3 2 0 3 0 0 1 0 0 3 0 0 0 1 3 1 2 1 0 3 0 1 1
 3 1 0 3 2 0 0 3 0 3 0 0 3 1 0 1 0 0 1 0 1 0 1 1 3 3 0 0 0 2 0 0 1 1 1 3 3
 0 2 3 2 3 0 1 3 3 3 3 2 1 0 0 0 2 3 0 1 1 1 1 3 1 0 3 0 0 3 0 2 0 3 1 1 0
 1 0 2 3 3 0 0 3 2 0 0 3 1 0 0 0 2 3 1 1 3 1 0 3 2 1 2 0 0 1 0 0 1 1 2 3 2
 0 1 0 0 0 0 1 0 0 0 0 1 1 3 3 3 3 0 3 1 2 3 1 0 3 0 2 3 2 3 2 0 0 2 1 1 3
 1 0 0 2 0 0 3 3 2 3 0 3 2 1 0 3 0 0 1 1 0 0 0 0 1 0 2 1 3 3 0 3 3 3 0 3 2
 0 0 1 1 0 0 1 0 0 2 1 1 0 0 0 1 1 1 2 0 2 0 1 3 1 0 0 2 3 0 0 3 0 3 3 1 1
 0 3 3 1 1 1 1 1 0 1 0 0 1 0 0 2 0 0 1 1 0 3 1 3 1 3 3 1 0 3 0 0 2 0 0 2 1
 0 3 1 2 1 2 0 0 1 0 0 0 0 3 0 3 0 3 3 2 0 3 3 0 3 3 3 1 3 0 1 1 3 3 3 1 2
 0 0 0 1 2 1 1 3 2 3 0 0 0 3 3 3 0 1 1 1 3 3 0 1 1 2 0 3 3 3 0 3 3 1 2 0 3
 1 3 3 3 0 1 3 3 3 3 0 3 3 0 0 0 0 1 1 3 0 0 1 1 0 0 2 3 0 1 3 1 0 1 3 3 2
 0 3 1 0 1 0 2 1 1 1 1 0 3 3 0 2 0 2 0 0 2 0 0 0 3 2 3 3 1 0 3 0 1 1 1 1 3
 3 1 2 3 1 1 0 3 3 0 0 0 3 2 0 3 1 0 1 3 3 1 1 0 3 3 1 3 3 0 2 0 3 3 3 3 3
 3 2 0

In [14]:
predicted_classes_np.shape

(1000,)

In [15]:
# Convert predictions to original array format
predictions_cnn = make_original_array(predicted_classes_np)

# Create a DataFrame and save to CSV
df_resnet = pd.DataFrame(predictions_cnn, columns=['ID', ' Class'])
df_resnet.to_csv('ResNet_Attempt_reducingLR_v2.csv', index=False)

In [19]:
df_resnet_0=pd.read_csv('ResNet_Attempt_reducingLR.csv')

In [20]:
df_resnet_1=pd.read_csv('ResNet_Attempt_2_ep7.csv')

In [21]:
# Assuming the predicted classes are in a column named 'Class'
differences = (df_resnet_0[' Class'] != df_resnet[' Class']).sum()
total = len(df_resnet)

print(f"Total Predictions: {total}")
print(f"Different Predictions: {differences}")
print(f"Percentage Difference: {(differences / total) * 100:.2f}%")

Total Predictions: 1000
Different Predictions: 122
Percentage Difference: 12.20%


In [19]:
# Get the count of each class
class_counts = df_resnet[' Class'].value_counts()

# Display the counts
print("Counts for each class in latest ResNet:")
print(class_counts)

# Optional: Convert to dictionary for further use
class_counts_dict = class_counts.to_dict()


Counts for each class in latest ResNet:
 Class
0    360
3    296
1    232
2    112
Name: count, dtype: int64


In [20]:
# Get the count of each class
class_counts = df_resnet_0[' Class'].value_counts()

# Display the counts
print("Counts for each class in first ResNet:")
print(class_counts)

# Optional: Convert to dictionary for further use
class_counts_dict = class_counts.to_dict()

Counts for each class in first ResNet:
 Class
0    342
3    299
1    261
2     98
Name: count, dtype: int64


In [29]:
# Get the count of each class
class_counts = df_resnet_1[' Class'].value_counts()

# Display the counts
print("Counts for each class in 2nd ResNet:")
print(class_counts)

# Optional: Convert to dictionary for further use
class_counts_dict = class_counts.to_dict()

Counts for each class in 2nd ResNet:
 Class
0    359
1    268
3    254
2    119
Name: count, dtype: int64


In [35]:
print( np.array(df_cnn_augb[1:]))

[[   2    0]
 [   3    3]
 [   4    3]
 ...
 [ 998    3]
 [ 999    3]
 [1000    0]]


In [36]:
print(df_resnet)

       ID   Class
0       1       3
1       2       0
2       3       3
3       4       3
4       5       0
..    ...     ...
995   996       2
996   997       1
997   998       3
998   999       1
999  1000       0

[1000 rows x 2 columns]


### LR= 1.5e-4 and epochs = 12

In [12]:
# Shuffle the data
np.random.seed(42)
num_samples = images.shape[0]

indices = np.arange(num_samples)
np.random.shuffle(indices)
shuffled_images = images[indices]
shuffled_labels = labels[indices]

# Split into training and validation sets
baseline1Idxs = int(num_samples * 0.8)  # 80% for training, 20% for validation
split_index = int(baseline1Idxs * 0.8)

train_images = shuffled_images[:split_index]
train_labels = shuffled_labels[:split_index]

val_images = shuffled_images[split_index:]
val_labels = shuffled_labels[split_index:]

# Verify shapes
print(f"Train Images: {train_images.shape}, Train Labels: {train_labels.shape}")
print(f"Validation Images: {val_images.shape}, Validation Labels: {val_labels.shape}")

Train Images: (62384, 28, 28), Train Labels: (62384,)
Validation Images: (35093, 28, 28), Validation Labels: (35093,)


In [14]:
# Determine whether to use GPU or CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from sklearn.metrics import roc_auc_score
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Custom Dataset
class CustomDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label


# Data Augmentation
augment_gen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    shear_range=15,
    horizontal_flip=True,
    fill_mode="nearest"
)

# Augment images with labels 1 and 2
mask1 = np.isin(train_labels, [1, 2])
filtered_images_1_2 = train_images[mask1]
filtered_labels_1_2 = train_labels[mask1]

augmented_images = []
augmented_labels = []

for img, lbl in zip(filtered_images_1_2, filtered_labels_1_2):
    img = np.expand_dims(img, axis=-1)  # Add channel dimension
    img = np.expand_dims(img, axis=0)   # Add batch dimension
    for _ in range(10):  # Generate 10 augmented samples per image
        augmented = next(augment_gen.flow(img, batch_size=1))[0]
        augmented_images.append(augmented)
        augmented_labels.append(lbl)

# Augment images with labels 0 and 3
mask2 = np.isin(train_labels, [0, 3])
filtered_images_0_3 = train_images[mask2]
filtered_labels_0_3 = train_labels[mask2]

for img, lbl in zip(filtered_images_0_3, filtered_labels_0_3):
    img = np.expand_dims(img, axis=-1)  # Add channel dimension
    img = np.expand_dims(img, axis=0)   # Add batch dimension
    for _ in range(2):  # Generate 2 augmented samples per image
        augmented = next(augment_gen.flow(img, batch_size=1))[0]
        augmented_images.append(augmented)
        augmented_labels.append(lbl)

# Combine augmented data with original training data
augmented_images = np.array(augmented_images)
augmented_labels = np.array(augmented_labels)
train_images_cnn = np.expand_dims(train_images, axis=-1)  # Add channel dimension to training images
train_images_cnn = np.concatenate((train_images_cnn, augmented_images), axis=0)
train_labels_cnn = np.concatenate((train_labels, augmented_labels), axis=0)

# Convert labels to class indices
train_labels_cnn = train_labels_cnn.astype(int)
val_labels = val_labels.astype(int)

# PyTorch Dataset and DataLoader
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize
])

train_dataset = CustomDataset(train_images_cnn, train_labels_cnn, transform=transform)
val_dataset = CustomDataset(val_images, val_labels, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Bottleneck Block for ResNet-50
class Bottleneck(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(out_channels, out_channels * 4, kernel_size=1, stride=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample

    def forward(self, x):
        identity = x
        if self.downsample is not None:
            identity = self.downsample(x)

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(x)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.conv3(out)
        out = self.bn3(out)

        out += identity
        out = self.relu(out)
        return out


# ResNet-50 for 28x28 Inputs
class ResNet28x28(nn.Module):
    def __init__(self, block, layers, num_classes=4):
        super(ResNet28x28, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=False)  # Adjusted for 28x28
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=1)  # No stride for 28x28 input
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * 4, num_classes)

    def _make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if stride != 1 or self.in_channels != out_channels * 4:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, out_channels * 4, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * 4),
            )

        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels * 4
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, out_channels))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x


# Compute Class Weights for Balanced Loss
class_weights = compute_class_weight('balanced', classes=np.unique(train_labels_cnn), y=train_labels_cnn)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)


# Temperature Scaled Cross Entropy Loss
class TemperatureScaledCrossEntropyLoss(nn.Module):
    def __init__(self, temperature=2.0):
        super(TemperatureScaledCrossEntropyLoss, self).__init__()
        self.temperature = temperature
        self.criterion = nn.CrossEntropyLoss(weight=class_weights)

    def forward(self, logits, targets):
        scaled_logits = logits / self.temperature
        return self.criterion(scaled_logits, targets)


# Training Loop with AUC Calculation
def train_resnet28(model, train_loader, val_loader, epochs=10):
    criterion = TemperatureScaledCrossEntropyLoss(temperature=2.0)
    optimizer = optim.Adam(model.parameters(), lr=1.5e-4, weight_decay=1e-5)

    for epoch in range(epochs):
        model.train()
        train_loss = 0
        train_correct = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            train_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            _, preds = torch.max(outputs, 1)
            train_correct += (preds == labels).sum().item()

        # Validation Phase
        model.eval()
        val_loss = 0
        val_correct = 0
        all_preds = []
        all_labels = []

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                _, preds = torch.max(outputs, 1)
                val_correct += (preds == labels).sum().item()

                # Store predicted probabilities and true class indices
                all_preds.extend(torch.softmax(outputs, dim=1).cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        # Calculate AUC
        val_auc = roc_auc_score(
            np.eye(4)[np.array(all_labels)],  # Convert class indices to one-hot encoded for AUC
            np.array(all_preds),
            multi_class='ovr'
        )

        # Calculate Metrics
        train_accuracy = train_correct / len(train_loader.dataset)
        val_accuracy = val_correct / len(val_loader.dataset)

        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, "
              f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}, Val AUC: {val_auc:.4f}")


# Instantiate Model and Train
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ResNet28x28(Bottleneck, [3, 4, 6, 3], num_classes=4).to(device)

train_resnet28(model, train_loader, val_loader, epochs=12)


RuntimeError: Given groups=1, weight of size [64, 64, 3, 3], expected input[32, 256, 28, 28] to have 64 channels, but got 256 channels instead

### VGG Net Kinda implementation

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from sklearn.metrics import roc_auc_score
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Custom Dataset
class CustomDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label


# Data Augmentation
augment_gen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    shear_range=15,
    horizontal_flip=True,
    fill_mode="nearest"
)

# Augment images with labels 1 and 2
mask1 = np.isin(train_labels, [1, 2])
filtered_images_1_2 = train_images[mask1]
filtered_labels_1_2 = train_labels[mask1]

augmented_images = []
augmented_labels = []

for img, lbl in zip(filtered_images_1_2, filtered_labels_1_2):
    img = np.expand_dims(img, axis=-1)  # Add channel dimension
    img = np.expand_dims(img, axis=0)   # Add batch dimension
    for _ in range(10):  # Generate 10 augmented samples per image
        augmented = next(augment_gen.flow(img, batch_size=1))[0]
        augmented_images.append(augmented)
        augmented_labels.append(lbl)

# Augment images with labels 0 and 3
mask2 = np.isin(train_labels, [0, 3])
filtered_images_0_3 = train_images[mask2]
filtered_labels_0_3 = train_labels[mask2]

for img, lbl in zip(filtered_images_0_3, filtered_labels_0_3):
    img = np.expand_dims(img, axis=-1)  # Add channel dimension
    img = np.expand_dims(img, axis=0)   # Add batch dimension
    for _ in range(2):  # Generate 2 augmented samples per image
        augmented = next(augment_gen.flow(img, batch_size=1))[0]
        augmented_images.append(augmented)
        augmented_labels.append(lbl)

# Combine augmented data with original training data
augmented_images = np.array(augmented_images)
augmented_labels = np.array(augmented_labels)
train_images_cnn = np.expand_dims(train_images, axis=-1)  # Add channel dimension to training images
train_images_cnn = np.concatenate((train_images_cnn, augmented_images), axis=0)
train_labels_cnn = np.concatenate((train_labels, augmented_labels), axis=0)

# Convert labels to class indices
train_labels_cnn = train_labels_cnn.astype(int)
val_labels = val_labels.astype(int)

# PyTorch Dataset and DataLoader
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize
])

train_dataset = CustomDataset(train_images_cnn, train_labels_cnn, transform=transform)
val_dataset = CustomDataset(val_images, val_labels, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


# VGG-like Model
class ImprovedCNN(nn.Module):
    def __init__(self, num_classes=4):
        super(ImprovedCNN, self).__init__()
        self.features = nn.Sequential(
            # Convolutional Layer 1
            nn.Conv2d(1, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Output: 64x14x14

            # Convolutional Layer 2
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Output: 128x7x7

            # Convolutional Layer 3
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)  # Output: 256x3x3
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),  # Flatten the output of convolutional layers
            nn.Linear(256 * 3 * 3, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x


# Compute Class Weights for Balanced Loss
class_weights = compute_class_weight('balanced', classes=np.unique(train_labels_cnn), y=train_labels_cnn)
class_weights = torch.tensor(class_weights, dtype=torch.float).to("cuda" if torch.cuda.is_available() else "cpu")


# Training Loop with AUC Calculation
def train_vgg(model, train_loader, val_loader, device, epochs=10):
    criterion = nn.CrossEntropyLoss(weight=class_weights)
    optimizer = optim.Adam(model.parameters(), lr=8e-5)

    for epoch in range(epochs):
        model.train()
        train_loss = 0
        train_correct = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            train_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            _, preds = torch.max(outputs, 1)
            train_correct += (preds == labels).sum().item()

        # Validation
        model.eval()
        val_loss = 0
        val_correct = 0
        all_preds = []
        all_labels = []

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                val_loss += criterion(outputs, labels).item()

                _, preds = torch.max(outputs, 1)
                val_correct += (preds == labels).sum().item()
                all_preds.extend(torch.softmax(outputs, dim=1).cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        # Calculate AUC
        val_auc = roc_auc_score(
            np.eye(4)[np.array(all_labels)],  # One-hot encode true labels
            np.array(all_preds),
            multi_class='ovr'
        )

        # Metrics
        train_accuracy = train_correct / len(train_loader.dataset)
        val_accuracy = val_correct / len(val_loader.dataset)

        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, "
              f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}, Val AUC: {val_auc:.4f}")


# Instantiate the model and train
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ImprovedCNN(num_classes=4).to(device)

train_vgg(model, train_loader, val_loader, device, epochs=20)


Epoch 1/20, Train Loss: 6530.0767, Train Accuracy: 0.6981, Val Loss: 517.0303, Val Accuracy: 0.8465, Val AUC: 0.9360
Epoch 2/20, Train Loss: 5018.5268, Train Accuracy: 0.7751, Val Loss: 414.7830, Val Accuracy: 0.8679, Val AUC: 0.9583
Epoch 3/20, Train Loss: 4475.6343, Train Accuracy: 0.7994, Val Loss: 394.4203, Val Accuracy: 0.8740, Val AUC: 0.9589
Epoch 4/20, Train Loss: 4054.8730, Train Accuracy: 0.8180, Val Loss: 374.5135, Val Accuracy: 0.8891, Val AUC: 0.9634
Epoch 5/20, Train Loss: 3693.6781, Train Accuracy: 0.8347, Val Loss: 402.5338, Val Accuracy: 0.8808, Val AUC: 0.9632
Epoch 6/20, Train Loss: 3339.1440, Train Accuracy: 0.8505, Val Loss: 353.4662, Val Accuracy: 0.8950, Val AUC: 0.9656
Epoch 7/20, Train Loss: 2996.5623, Train Accuracy: 0.8653, Val Loss: 369.1341, Val Accuracy: 0.8928, Val AUC: 0.9670
Epoch 8/20, Train Loss: 2660.1295, Train Accuracy: 0.8796, Val Loss: 384.7228, Val Accuracy: 0.8886, Val AUC: 0.9664
Epoch 9/20, Train Loss: 2349.5072, Train Accuracy: 0.8930, Val L

In [15]:
import torch
import numpy as np
from torchvision import transforms

# Assuming testimages is a NumPy array of shape (num_samples, 28, 28)

# Preprocess test images
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # Ensure same normalization as during training
])

# Convert testimages to PyTorch Tensor with proper dimensions
test_images_tensor = torch.stack([test_transform(img) for img in testimages])  # Shape: (num_samples, 1, 28, 28)

# Send to device (e.g., GPU or CPU)
test_images_tensor = test_images_tensor.to(device)

# Ensure the model is in evaluation mode
model.eval()

# Make predictions
with torch.no_grad():
    logits = model(test_images_tensor)  # Shape: (num_samples, num_classes)
    probabilities = torch.softmax(logits, dim=1)  # Convert logits to probabilities
    predicted_classes = torch.argmax(probabilities, dim=1)  # Get class with highest probability

# Convert predictions to NumPy array
predicted_classes_np = predicted_classes.cpu().numpy()

# Print predictions
print("Predicted Classes:", predicted_classes_np)

Predicted Classes: [3 0 3 3 0 2 1 0 0 3 0 1 3 3 3 2 0 1 0 0 1 0 2 3 0 0 0 1 3 1 3 1 3 3 2 1 0
 3 1 0 1 0 0 0 0 0 1 1 1 3 1 0 1 0 0 0 0 1 0 1 1 3 3 2 0 0 0 0 0 1 0 1 1 3
 0 2 3 2 3 0 1 3 3 1 3 2 1 2 2 0 0 3 0 1 1 0 0 3 1 0 3 0 0 3 0 2 2 3 1 2 0
 1 0 2 3 3 0 0 0 2 0 0 3 1 0 0 0 0 3 1 1 3 1 0 1 2 1 2 0 0 1 0 0 1 1 2 3 2
 0 1 1 0 0 1 1 0 0 0 0 1 3 3 3 3 3 0 3 1 3 3 1 0 3 0 2 3 2 3 0 0 2 2 1 1 2
 2 0 0 2 0 0 3 3 2 3 0 3 2 1 2 3 0 0 1 1 0 1 0 0 1 3 2 1 1 3 0 3 1 3 0 3 3
 0 0 2 3 0 0 3 0 0 0 1 1 0 0 0 1 1 1 2 2 2 0 1 3 1 0 0 1 3 0 0 3 0 3 3 1 1
 0 1 3 1 1 0 1 1 1 0 0 1 1 0 0 2 0 0 1 1 0 3 1 3 1 3 2 1 0 3 0 0 2 0 0 2 1
 0 3 1 1 0 3 0 0 1 0 0 0 0 3 0 3 0 3 3 1 0 0 3 0 3 3 0 3 3 0 1 1 1 3 3 3 2
 0 0 2 1 2 0 1 3 3 1 0 0 0 3 3 3 2 1 1 0 3 3 0 1 1 2 0 3 3 3 0 3 3 0 2 1 3
 1 3 3 1 1 1 3 3 3 3 1 3 3 0 0 2 0 1 1 3 0 0 1 1 0 0 2 3 0 1 3 1 0 0 3 3 2
 0 3 1 0 1 0 2 1 3 1 1 0 3 3 0 1 0 2 0 0 0 0 0 0 3 2 3 3 1 3 3 0 1 1 1 1 3
 3 1 2 3 1 0 2 3 3 0 0 0 2 2 0 3 0 0 1 3 1 1 1 0 3 3 1 3 3 3 2 1 1 1 3 3 3
 3 2 0

In [16]:
df_resnet_1=pd.read_csv('ResNet_Attempt_2_ep7.csv')
df_resnet=pd.read_csv('ResNet_Attempt.csv')
df_resnet_2=pd.read_csv('ResNet_Attempt_1.5_ep12.csv')
# Convert predictions to original array format
predictions_cnn = make_original_array(predicted_classes_np)

# Create a DataFrame and save to CSV
df_vgg = pd.DataFrame(predictions_cnn, columns=['ID', ' Class'])
df_vgg.to_csv('VggNet.csv', index=False)


In [25]:
# Assuming the predicted classes are in a column named 'Class'
differences = (df_vgg[' Class'] != df_resnet[' Class']).sum()
total = len(df_resnet)

print(f"Total Predictions: {total}")
print(f"Different Predictions: {differences}")
print(f"Percentage Difference: {(differences / total) * 100:.2f}%")

Total Predictions: 1000
Different Predictions: 196
Percentage Difference: 19.60%


In [18]:
# Get the count of each class
class_counts = df_resnet[' Class'].value_counts()

# Display the counts
print("Counts for each class in latest ResNet:")
print(class_counts)

# Optional: Convert to dictionary for further use
class_counts_dict = class_counts.to_dict()


Counts for each class in latest ResNet:
 Class
0    342
3    299
1    261
2     98
Name: count, dtype: int64


In [19]:
# Get the count of each class
class_counts = df_resnet_1[' Class'].value_counts()

# Display the counts
print("Counts for each class in first ResNet:")
print(class_counts)

# Optional: Convert to dictionary for further use
class_counts_dict = class_counts.to_dict()

Counts for each class in first ResNet:
 Class
0    359
1    268
3    254
2    119
Name: count, dtype: int64


In [20]:
# Get the count of each class
class_counts = df_resnet_2[' Class'].value_counts()

# Display the counts
print("Counts for each class in 2nd ResNet:")
print(class_counts)

# Optional: Convert to dictionary for further use
class_counts_dict = class_counts.to_dict()

Counts for each class in 2nd ResNet:
 Class
0    318
3    298
1    277
2    107
Name: count, dtype: int64


In [22]:
# Get the count of each class
class_counts = df_vgg[' Class'].value_counts()

# Display the counts
print("Counts for each class in latest ResNet:")
print(class_counts)

# Optional: Convert to dictionary for further use
class_counts_dict = class_counts.to_dict()


Counts for each class in latest ResNet:
 Class
0    357
1    274
3    259
2    110
Name: count, dtype: int64


### Vision transformer updated

In [26]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from sklearn.metrics import roc_auc_score
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import timm


# Custom Dataset
class CustomDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label


# Data Augmentation
augment_gen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    shear_range=15,
    horizontal_flip=True,
    fill_mode="nearest"
)

# Augment images with labels 1 and 2
mask1 = np.isin(train_labels, [1, 2])
filtered_images_1_2 = train_images[mask1]
filtered_labels_1_2 = train_labels[mask1]

augmented_images = []
augmented_labels = []

for img, lbl in zip(filtered_images_1_2, filtered_labels_1_2):
    img = np.expand_dims(img, axis=-1)  # Add channel dimension
    img = np.expand_dims(img, axis=0)   # Add batch dimension
    for _ in range(10):  # Generate 10 augmented samples per image
        augmented = next(augment_gen.flow(img, batch_size=1))[0]
        augmented_images.append(augmented)
        augmented_labels.append(lbl)

# Augment images with labels 0 and 3
mask2 = np.isin(train_labels, [0, 3])
filtered_images_0_3 = train_images[mask2]
filtered_labels_0_3 = train_labels[mask2]

for img, lbl in zip(filtered_images_0_3, filtered_labels_0_3):
    img = np.expand_dims(img, axis=-1)  # Add channel dimension
    img = np.expand_dims(img, axis=0)   # Add batch dimension
    for _ in range(2):  # Generate 2 augmented samples per image
        augmented = next(augment_gen.flow(img, batch_size=1))[0]
        augmented_images.append(augmented)
        augmented_labels.append(lbl)

# Combine augmented data with original training data
augmented_images = np.array(augmented_images)
augmented_labels = np.array(augmented_labels)
train_images_vit = np.expand_dims(train_images, axis=-1)  # Add channel dimension to training images
train_images_vit = np.concatenate((train_images_vit, augmented_images), axis=0)
train_labels_vit = np.concatenate((train_labels, augmented_labels), axis=0)

# Convert labels to class indices
train_labels_vit = train_labels_vit.astype(int)
val_labels = val_labels.astype(int)

# PyTorch Dataset and DataLoader
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Grayscale(num_output_channels=3),  # Convert grayscale to 3 channels for ViT
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize
])

train_dataset = CustomDataset(train_images_vit, train_labels_vit, transform=transform)
val_dataset = CustomDataset(val_images, val_labels, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


# Vision Transformer Model for 28x28 Images
class ViTModel28x28(nn.Module):
    def __init__(self, num_classes=4):
        super(ViTModel28x28, self).__init__()
        self.vit = timm.create_model(
            'vit_base_patch16_224',
            pretrained=True,
            img_size=28,
            patch_size=7
        )
        self.vit.head = nn.Linear(self.vit.head.in_features, num_classes)

    def forward(self, x):
        return self.vit(x)


# Compute Class Weights for Balanced Loss
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class_weights = compute_class_weight('balanced', classes=np.unique(train_labels_vit), y=train_labels_vit)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)


# Training Loop with AUC Calculation
def train_vit(model, train_loader, val_loader, epochs=10):
    criterion = nn.CrossEntropyLoss(weight=class_weights)
    optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)

    for epoch in range(epochs):
        # Training Phase
        model.train()
        train_loss = 0
        train_correct = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            train_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            _, preds = torch.max(outputs, 1)
            train_correct += (preds == labels).sum().item()

        # Validation Phase
        model.eval()
        val_loss = 0
        val_correct = 0
        all_preds = []
        all_labels = []

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                val_loss += criterion(outputs, labels).item()

                _, preds = torch.max(outputs, 1)
                val_correct += (preds == labels).sum().item()
                all_preds.extend(torch.softmax(outputs, dim=1).cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        # Calculate AUC
        val_auc = roc_auc_score(
            np.eye(4)[np.array(all_labels)],  # Convert class indices to one-hot encoded for AUC
            np.array(all_preds),
            multi_class='ovr'
        )

        # Metrics
        train_accuracy = train_correct / len(train_loader.dataset)
        val_accuracy = val_correct / len(val_loader.dataset)

        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, "
              f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}, Val AUC: {val_auc:.4f}")


# Instantiate and Train Model
model = ViTModel28x28(num_classes=4).to(device)
train_vit(model, train_loader, val_loader, epochs=12)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Epoch 1/12, Train Loss: 6572.6255, Train Accuracy: 0.6929, Val Loss: 514.6851, Val Accuracy: 0.8535, Val AUC: 0.9343
Epoch 2/12, Train Loss: 5195.8592, Train Accuracy: 0.7628, Val Loss: 465.3332, Val Accuracy: 0.8498, Val AUC: 0.9502
Epoch 3/12, Train Loss: 4563.4953, Train Accuracy: 0.7930, Val Loss: 436.7499, Val Accuracy: 0.8678, Val AUC: 0.9526
Epoch 4/12, Train Loss: 4058.6413, Train Accuracy: 0.8154, Val Loss: 435.0876, Val Accuracy: 0.8705, Val AUC: 0.9506
Epoch 5/12, Train Loss: 3586.2547, Train Accuracy: 0.8368, Val Loss: 425.9187, Val Accuracy: 0.8711, Val AUC: 0.9563
Epoch 6/12, Train Loss: 3135.0203, Train Accuracy: 0.8575, Val Loss: 440.3009, Val Accuracy: 0.8624, Val AUC: 0.9542
Epoch 7/12, Train Loss: 2721.2417, Train Accuracy: 0.8757, Val Loss: 449.7716, Val Accuracy: 0.8686, Val AUC: 0.9562
Epoch 8/12, Train Loss: 2345.7958, Train Accuracy: 0.8937, Val Loss: 495.6981, Val Accuracy: 0.8713, Val AUC: 0.9537
Epoch 9/12, Train Loss: 2027.4306, Train Accuracy: 0.9078, Val L